In [2]:
%load_ext autoreload
%autoreload 2

import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_recommenders as tfrs

from importlib import reload
reload(tfrs)

import os
import pprint
import tempfile
from typing import Dict, Text
import numpy as np

import pandas as pd
import numpy as np
import swifter
import re
import json
import ast
import pprint

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
#tf.config.list_physical_devices('GPU')
#tf.config.list_physical_devices('CPU')
print(tf.test.is_gpu_available())

False


### Product data

In [4]:
# Load product catalog data
product = list()
with open('../dataset/retailrocket_processed_data/clean_data/product_catalog_Retail_API.json') as file:
    for i in file:
        product.append(json.loads(i))
product_df = pd.DataFrame(product).drop(['type', 'primaryProductId','availability'],1)
product_df.head(5)

,id,title,categories,description,priceInfo
0,1,1299107,"[[1114.0, 113.0]]",513325 1020281 1204938 172646 72261 30603 8980...,"{'currencyCode': 'USD', 'price': 6120, 'origin..."
1,2,1260440,"[[1305.0, 1214.0]]",822092 325894 504272 147366 343631 648485 n600...,"{'currencyCode': 'USD', 'price': 41040, 'origi..."
2,3,354388,"[[1171.0, 938.0]]",138228 150169 1182824 327918 261419,"{'currencyCode': 'USD', 'price': 1560, 'origin..."
3,5,1000017,"[[1061.0, 1259.0]]",188678 316731 30603 341599 1208937 1300184,"{'currencyCode': 'USD', 'price': 100, 'origina..."
4,6,303099,"[[1091.0, 573.0]]",1236550 662786 609354 662786 302095 639502 123...,"{'currencyCode': 'USD', 'price': 41760, 'origi..."


In [5]:
product_df = product_df.rename({'id': 'item_id', 'title': 'item_title', 'categories': 'category_id'}, axis=1)
product_df['price'] = product_df['priceInfo'].swifter.apply(lambda x: x['price'])
product_df['original_price'] = product_df['priceInfo'].swifter.apply(lambda x: x['originalPrice'])
product_df.head(4)

Pandas Apply:   0%|          | 0/417053 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/417053 [00:00<?, ?it/s]

,item_id,item_title,category_id,description,priceInfo,price,original_price
0,1,1299107,"[[1114.0, 113.0]]",513325 1020281 1204938 172646 72261 30603 8980...,"{'currencyCode': 'USD', 'price': 6120, 'origin...",6120,6240
1,2,1260440,"[[1305.0, 1214.0]]",822092 325894 504272 147366 343631 648485 n600...,"{'currencyCode': 'USD', 'price': 41040, 'origi...",41040,41040
2,3,354388,"[[1171.0, 938.0]]",138228 150169 1182824 327918 261419,"{'currencyCode': 'USD', 'price': 1560, 'origin...",1560,1560
3,5,1000017,"[[1061.0, 1259.0]]",188678 316731 30603 341599 1208937 1300184,"{'currencyCode': 'USD', 'price': 100, 'origina...",100,100


In [6]:
product_df['category_id'] = product_df['category_id'].swifter.apply(lambda x: x[0])
product_df['price'] = product_df['price'].swifter.apply(lambda x: int(x))
product_df['original_price'] = product_df['original_price'].swifter.apply(lambda x: int(x))

Pandas Apply:   0%|          | 0/417053 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/417053 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/417053 [00:00<?, ?it/s]

In [7]:
# Prepare data in tensors
product = tf.data.Dataset.from_tensor_slices({'item_id': product_df['item_id'].tolist(),'item_title': product_df['item_title'].tolist(),
                                             'category_id': product_df['category_id'].tolist(), 'description': product_df['description'].tolist(),
                                             'price': product_df['price'].tolist(), 'original_price': product_df['original_price'].tolist()})

In [8]:
for i in product.take(2).as_numpy_iterator():
    pprint.pprint(i)

{'category_id': array([b'1114.0', b'113.0'], dtype=object),
 'description': b'513325 1020281 1204938 172646 72261 30603 898004 140719 3028'
                b'48 1204938 190728 532620 572715 1128577 162103 1097148 28873'
                b'1 418473 150169 855390 836649 113294 435459 1103229 1019257 '
                b'1272915 707081 13541 1161933 610340 804917',
 'item_id': b'1',
 'item_title': b'1299107',
 'original_price': 6240,
 'price': 6120}
{'category_id': array([b'1305.0', b'1214.0'], dtype=object),
 'description': b'822092 325894 504272 147366 343631 648485 n6000.000 n72.000 '
                b'504272 147366 343631 648485 n6000.000 639502 n72.000 936990 '
                b'949802 1113810 504272 653611 325894 1262898 796845 161652 15'
                b'0169 353870 1309799 1030323 608362 680618 1128577 636684 108'
                b'1984 703408',
 'item_id': b'2',
 'item_title': b'1260440',
 'original_price': 41040,
 'price': 41040}


In [9]:
# Item - Category mapping
item_category_map = dict(zip(product_df['item_id'].tolist(),product_df['category_id'].tolist()))
item_category_map['3']

['1171.0', '938.0']

### User events data

In [10]:
# Load user events data
with open('../dataset/microsoft_training_ClickRate_data') as file:
    training = file.readlines()
df = pd.DataFrame(training,columns = ['original'])

In [11]:
df['user_id'] = df['original'].swifter.apply(lambda x: str(x.split('\t')[1]))
df['item_id'] = df['original'].swifter.apply(lambda x: str(x.split('\t')[2]))
df['category_id'] = df['original'].swifter.apply(lambda x: [np.int(float(x.split('\t')[3]))]) # category = integer
df['timestamp'] = df['original'].swifter.apply(lambda x: int(x.split('\t')[4]))

Pandas Apply:   0%|          | 0/909817 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/909817 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/909817 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/909817 [00:00<?, ?it/s]

In [12]:
df['category_id'] = df['item_id'].swifter.apply(lambda x: item_category_map[x])
df['category_id'] = df['category_id'].swifter.apply(lambda x: x[0])
df.head(3)

Pandas Apply:   0%|          | 0/909817 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/909817 [00:00<?, ?it/s]

,original,user_id,item_id,category_id,timestamp
0,1\t2\t325215\t299.0\t1613850013\t325215\t299.0...,2,325215,299.0,1613850013
1,"1\t2\t259884\t299.0\t1613850212\t325215,325215...",2,259884,299.0,1613850212
2,"1\t2\t216305\t299.0\t1613850468\t325215,325215...",2,216305,299.0,1613850468


In [13]:
# Prepare data in tensors
user_events = tf.data.Dataset.from_tensor_slices({'user_id': df['user_id'].tolist(),'item_id': df['item_id'].tolist(),
                                             'category_id': df['category_id'].tolist(), 'timestamp': df['timestamp'].tolist()})
for i in user_events.take(2).as_numpy_iterator():
    print(i)

{'user_id': b'2', 'item_id': b'325215', 'category_id': b'299.0', 'timestamp': 1613850013}
{'user_id': b'2', 'item_id': b'259884', 'category_id': b'299.0', 'timestamp': 1613850212}


### Input data preparation

In [15]:
ratings = user_events.map(lambda x: {
    'user_id': x['user_id'],
    'item_id': x['item_id'],
    'category_id': x['category_id'],
    'timestamp': x['timestamp'],
}) # Retain only useful features
#movies = product.map(lambda x: x['item_id'])
movies = product.map(lambda x: {
    'item_id': x['item_id'],
    'category_id': x['category_id'],
    'description': x['description'],
    'item_title': x['item_title'],
    'original_price': x['original_price'],
    'price': x['price']
})

In [16]:
for i in ratings.take(2).as_numpy_iterator():
    pprint.pprint(i)

{'category_id': b'299.0',
 'item_id': b'325215',
 'timestamp': 1613850013,
 'user_id': b'2'}
{'category_id': b'299.0',
 'item_id': b'259884',
 'timestamp': 1613850212,
 'user_id': b'2'}


In [17]:
for i in movies.take(2).as_numpy_iterator():
    pprint.pprint(i)

{'category_id': array([b'1114.0', b'113.0'], dtype=object),
 'description': b'513325 1020281 1204938 172646 72261 30603 898004 140719 3028'
                b'48 1204938 190728 532620 572715 1128577 162103 1097148 28873'
                b'1 418473 150169 855390 836649 113294 435459 1103229 1019257 '
                b'1272915 707081 13541 1161933 610340 804917',
 'item_id': b'1',
 'item_title': b'1299107',
 'original_price': 6240,
 'price': 6120}
{'category_id': array([b'1305.0', b'1214.0'], dtype=object),
 'description': b'822092 325894 504272 147366 343631 648485 n6000.000 n72.000 '
                b'504272 147366 343631 648485 n6000.000 639502 n72.000 936990 '
                b'949802 1113810 504272 653611 325894 1262898 796845 161652 15'
                b'0169 353870 1309799 1030323 608362 680618 1128577 636684 108'
                b'1984 703408',
 'item_id': b'2',
 'item_title': b'1260440',
 'original_price': 41040,
 'price': 41040}


### user embeddings

In [18]:
user_id_lookup = tf.keras.layers.experimental.preprocessing.StringLookup()
user_id_lookup.adapt(ratings.map(lambda x: x["user_id"]))

In [23]:
max_timestamp = ratings.map(lambda x: x["timestamp"]).reduce(
    tf.cast(0, tf.int32), tf.maximum).numpy().max()
min_timestamp = ratings.map(lambda x: x["timestamp"]).reduce(
    np.int32(1e9), tf.minimum).numpy().min()
timestamp_buckets = np.linspace(
    min_timestamp, max_timestamp, num=1000)
print(f"Buckets: {timestamp_buckets[:3]}")

Buckets: [1.00000000e+09 1.00061491e+09 1.00122983e+09]


### item embeddings

In [48]:
movie_id_lookup = tf.keras.layers.experimental.preprocessing.StringLookup()
movie_id_lookup.adapt(movies.map(lambda x: x["item_id"]))
print(f"Vocabulary: {movie_title_lookup.get_vocabulary()[:3]}")

Vocabulary: ['', '[UNK]', '713045']


In [49]:
movie_title_lookup = tf.keras.layers.experimental.preprocessing.StringLookup()
movie_title_lookup.adapt(movies.map(lambda x: x["item_title"]))
print(f"Vocabulary: {movie_title_lookup.get_vocabulary()[:3]}")

Vocabulary: ['', '[UNK]', '713045']


In [50]:
description_text = tf.keras.layers.experimental.preprocessing.TextVectorization()
description_text.adapt(movies.map(lambda x: x["description"]))

In [51]:
price_normalization = tf.keras.layers.experimental.preprocessing.Normalization()
price_normalization.adapt(movies.map(lambda x: x["price"]).batch(1024))

### user model

In [29]:
class UserModel(tf.keras.Model):

  def __init__(self):
    super().__init__()

    self.user_embedding = tf.keras.Sequential([
        user_id_lookup,
        tf.keras.layers.Embedding(user_id_lookup.vocab_size(), 32),
    ])
    self.timestamp_embedding = tf.keras.Sequential([
      tf.keras.layers.experimental.preprocessing.Discretization(timestamp_buckets.tolist()),
      tf.keras.layers.Embedding(len(timestamp_buckets) + 2, 32)
    ])
    self.normalized_timestamp = tf.keras.layers.experimental.preprocessing.Normalization()

  def call(self, inputs):

    # Take the input dictionary, pass it through each input layer,
    # and concatenate the result.
    return tf.concat([
        self.user_embedding(inputs["user_id"]),
        self.timestamp_embedding(inputs["timestamp"]),
        self.normalized_timestamp(inputs["timestamp"])
    ], axis=1)

In [44]:
user_model = UserModel()

user_model.normalized_timestamp.adapt(
    ratings.map(lambda x: x["timestamp"]).batch(128))

for row in ratings.batch(1).take(1):
    print(f"Computed representations: {user_model(row)[0, :3]}")

Computed representations: [-0.02055042 -0.03011694 -0.00614251]


### Movie model

In [61]:
class MovieModel(tf.keras.Model):

  def __init__(self):
    super().__init__()

    max_tokens = 500_000

    self.id_embedding = tf.keras.Sequential([
      movie_id_lookup,
      tf.keras.layers.Embedding(movie_id_lookup.vocab_size(), 32)
    ])
    self.title_embedding = tf.keras.Sequential([
      movie_title_lookup,
      tf.keras.layers.Embedding(movie_title_lookup.vocab_size(), 32)
    ])
    self.description_text_embedding = tf.keras.Sequential([
      tf.keras.layers.experimental.preprocessing.TextVectorization(max_tokens=max_tokens),
      tf.keras.layers.Embedding(max_tokens, 32, mask_zero=True),
      # We average the embedding of individual words to get one embedding vector
      # per title.
      tf.keras.layers.GlobalAveragePooling1D(),
    ])
    self.normalized_price = tf.keras.layers.experimental.preprocessing.Normalization()

  def call(self, inputs):
    return tf.concat([
        self.id_embedding(inputs["item_id"]),
        self.title_embedding(inputs["item_title"]),
        self.description_text_embedding(inputs["description"]),
        self.normalized_price(inputs["price"])
    ], axis=1)

In [62]:
movie_model = MovieModel()

movie_model.description_text_embedding.layers[0].adapt(movies.map(lambda x: x["description"]))
movie_model.normalized_price.adapt(movies.map(lambda x: x["price"]).batch(128))

In [63]:
for row in movies.batch(1).take(1):
    print(f"Computed representations: {movie_model(row)[0, :4]}")

Computed representations: [ 0.01144432 -0.04837172 -0.02353926 -0.01859734]


In [66]:
movie_model(row).shape

TensorShape([1, 97])